# University of Michigan 
# Master of Applied Data Science
### SIAD 699: Capstone
Team: James Yoon (jamyoon), Mario Feliciano (felicma), and James Tuccori (jtuccori)

Date: April 2023
____
# Social Media Sentiment & Predicting Trading Signals 

#### THe purpose of this notebook is to run through the different paratmers for the timestep, neurons, droprate, and learning rate for our LSTM model and save the file for our Statistical Assesment Notebook. 

##### This took us over 70 hours to complete and even burned a GPU FAN in our on our teammates computer (lol) just to be able to process all these different parameter combinations


In [1]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from adspy_shared_utilities import plot_class_regions_for_classifier_subplot
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from adspy_shared_utilities import plot_decision_tree
from adspy_shared_utilities import plot_class_regions_for_classifier
from adspy_shared_utilities import plot_decision_tree
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from adspy_shared_utilities import plot_decision_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.ensemble import VotingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import ExtraTreesClassifier
import pickle
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix
from sklearn.inspection import permutation_importance
from sklearn.neural_network import MLPClassifier
from finta import TA

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from sklearn.model_selection import TimeSeriesSplit

In [3]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [4]:
INDICATORS = ['RSI', 'MACD', 'STOCH','ADL', 'ATR', 'MOM', 'MFI', 'ROC', 'OBV', 'CCI', 'EMV', 'VORTEX']
Df_hourly_merge = pd.read_csv('assets/PCHourly2019202_ActualLabel.csv')

def _exponential_smooth(data, alpha):
    """
    Function that exponentially smooths dataset so values are less 'rigid'
    :param alpha: weight factor to weight recent values more
    """
    
    return data.ewm(alpha=alpha).mean()

Indicatordata = _exponential_smooth(Df_hourly_merge[['Close', 'Open','High','Low','Volume']], 0.65)
def _get_indicator_data(data):
    """
    Function that uses the finta API to calculate technical indicators used as the features
    :return:
    """

    for indicator in INDICATORS:
        ind_data = eval('TA.' + indicator + '(data)')
        if not isinstance(ind_data, pd.DataFrame):
            ind_data = ind_data.to_frame()
        data = data.merge(ind_data, left_index=True, right_index=True)
    data.rename(columns={"14 period EMV.": '14 period EMV'}, inplace=True)

    # Also calculate moving averages for features
    data['ema50'] = data['Close'] / data['Close'].ewm(50).mean()
    data['ema21'] = data['Close'] / data['Close'].ewm(21).mean()
    data['ema15'] = data['Close'] / data['Close'].ewm(14).mean()
    data['ema5'] = data['Close'] / data['Close'].ewm(5).mean()

    # Instead of using the actual volume value (which changes over time), we normalize it with a moving volume average
    data['normVol'] = data['Volume'] / data['Volume'].ewm(5).mean()

    
    return data

Indicatordatafinal = _get_indicator_data(Indicatordata)

Indicatordatafinal = Indicatordatafinal.drop(['Close', 'Open', 'High', 'Low', 'Volume'], axis = 1)
Df_hourly_merge2 = pd.merge(Df_hourly_merge, Indicatordatafinal, left_index=True, right_index=True)

Df_hourly_merge2 = Df_hourly_merge2.drop(['time','hour','Open Time','_merge','Signal','Position', 'Signal35JMJ', 'Position35JMJ', 'Ignore'], axis = 1)




def binary(value):
  if value > 0:
    return 1
  else:
    return 0

columns = ['favorites', 'retweets', 'number_of_followers', 'following',
       'followers_following_ratio', '2x_retweets_+_favorites', 'polarity',
       'W1 Score', 'Bull_ratio', 'W Score With Bull Ratio', 'Open', 'High',
       'Low', 'Close', 'Volume', 'Quote Asset Volume', 'Number of Trades',
       'TB Base Volume', 'TB Quote Volume', '3MovingAverage', '5MovingAverage',
       'JMJ_3HMoving_averages', 'JMJ_5HMoving_averages', 
       '14 period RSI', 'MACD', 'SIGNAL', '14 period STOCH %K', 'MFV',
       '14 period ATR', 'MOM', '14 period MFI', 'ROC', 'OBV', '20 period CCI',
       '14 period EMV', 'VIm', 'VIp', 'ema50', 'ema21', 'ema15', 'ema5']


for column in columns: 
    Df_hourly_merge2['Binary{}'.format(column)]  = (Df_hourly_merge2[column].shift(1) - Df_hourly_merge2[column]).apply(binary)
    

In [5]:
Df_hourly_merge2.columns

Index(['favorites', 'retweets', 'number_of_followers', 'following',
       'followers_following_ratio', '2x_retweets_+_favorites', 'polarity',
       'W1 Score', 'Bull_ratio', 'W Score With Bull Ratio', 'Open', 'High',
       'Low', 'Close', 'Volume', 'Quote Asset Volume', 'Number of Trades',
       'TB Base Volume', 'TB Quote Volume', '3MovingAverage', '5MovingAverage',
       'JMJ_3HMoving_averages', 'JMJ_5HMoving_averages', 'Actual_Label',
       '14 period RSI', 'MACD', 'SIGNAL', '14 period STOCH %K', 'MFV',
       '14 period ATR', 'MOM', '14 period MFI', 'ROC', 'OBV', '20 period CCI',
       '14 period EMV', 'VIm', 'VIp', 'ema50', 'ema21', 'ema15', 'ema5',
       'normVol', 'Binaryfavorites', 'Binaryretweets',
       'Binarynumber_of_followers', 'Binaryfollowing',
       'Binaryfollowers_following_ratio', 'Binary2x_retweets_+_favorites',
       'Binarypolarity', 'BinaryW1 Score', 'BinaryBull_ratio',
       'BinaryW Score With Bull Ratio', 'BinaryOpen', 'BinaryHigh',
       'Binary

In [6]:
Df_hourly_merge2 = Df_hourly_merge2[['favorites', 'retweets', 'number_of_followers', 'following',
       'followers_following_ratio', '2x_retweets_+_favorites', 'polarity',
       'W1 Score', 'Bull_ratio', 'W Score With Bull Ratio', 'Open', 'High',
       'Low', 'Close', 'Volume', 'Quote Asset Volume', 'Number of Trades',
       'TB Base Volume', 'TB Quote Volume', '3MovingAverage', '5MovingAverage',
       'JMJ_3HMoving_averages', 'JMJ_5HMoving_averages',
       '14 period RSI', 'MACD', 'SIGNAL', '14 period STOCH %K', 'MFV',
       '14 period ATR', 'MOM', '14 period MFI', 'ROC', 'OBV', '20 period CCI',
       '14 period EMV', 'VIm', 'VIp', 'ema50', 'ema21', 'ema15', 'ema5',
       'normVol', 'Binaryfavorites', 'Binaryretweets',
       'Binarynumber_of_followers', 'Binaryfollowing',
       'Binaryfollowers_following_ratio', 'Binary2x_retweets_+_favorites',
       'Binarypolarity', 'BinaryW1 Score', 'BinaryBull_ratio',
       'BinaryW Score With Bull Ratio', 'BinaryOpen', 'BinaryHigh',
       'BinaryLow', 'BinaryClose', 'BinaryVolume', 'BinaryQuote Asset Volume',
       'BinaryNumber of Trades', 'BinaryTB Base Volume',
       'BinaryTB Quote Volume', 'Binary3MovingAverage', 'Binary5MovingAverage',
       'BinaryJMJ_3HMoving_averages', 'BinaryJMJ_5HMoving_averages',
       'Binary14 period RSI', 'BinaryMACD', 'BinarySIGNAL',
       'Binary14 period STOCH %K', 'BinaryMFV', 'Binary14 period ATR',
       'BinaryMOM', 'Binary14 period MFI', 'BinaryROC', 'BinaryOBV',
       'Binary20 period CCI', 'Binary14 period EMV', 'BinaryVIm', 'BinaryVIp',
       'Binaryema50', 'Binaryema21', 'Binaryema15', 'Binaryema5','Actual_Label']]
Df_hourly_merge2

,favorites,retweets,number_of_followers,following,followers_following_ratio,2x_retweets_+_favorites,polarity,W1 Score,Bull_ratio,W Score With Bull Ratio,...,BinaryOBV,Binary20 period CCI,Binary14 period EMV,BinaryVIm,BinaryVIp,Binaryema50,Binaryema21,Binaryema15,Binaryema5,Actual_Label
0,4.789144,0.764092,32454.039666,2574.079332,60.407159,6.317328,0.111223,0.016004,4.386364,0.070199,...,0,0,0,0,0,0,0,0,0,1.0
1,9.122449,1.253968,18279.126984,2235.809524,69.537416,11.630385,0.101236,0.180223,5.379310,0.969473,...,0,0,0,0,0,0,0,0,0,1.0
2,1.325521,0.289062,5000.981771,2029.653646,31.230965,1.903646,0.100770,0.096907,4.323529,0.418980,...,0,0,0,0,0,0,0,0,0,0.0
3,2.045699,0.303763,9224.862903,3983.379032,113.469624,2.653226,0.113049,0.042449,4.656250,0.197655,...,0,0,0,0,0,1,1,1,1,1.0
4,0.869674,0.102757,14964.082707,2214.571429,79.086570,1.075188,0.081205,0.007195,4.379310,0.031511,...,0,1,0,0,0,0,0,0,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25531,3.011905,0.392857,14180.595238,1765.121429,31.755382,3.797619,0.095683,0.135134,4.277778,0.578073,...,1,1,0,1,0,1,1,1,1,0.0
25532,14.400000,2.407143,16161.992857,2084.061905,182.260151,19.214286,0.082271,0.107210,3.648649,0.391172,...,1,1,1,0,1,1,1,1,1,1.0
25533,21.570806,3.270153,11254.296296,1814.954248,428.484448,28.111111,0.070485,0.165014,1.887324,0.311435,...,0,1,1,0,1,0,0,0,0,0.0
25534,7.505967,0.878282,10556.818616,2143.973747,140.745063,9.262530,0.079407,0.135342,3.945946,0.534052,...,1,1,1,0,1,1,1,1,1,1.0


In [8]:
Df_hourly_merge2.shape

(25536, 84)

In [9]:
Df_hourly_merge2

,favorites,retweets,number_of_followers,following,followers_following_ratio,2x_retweets_+_favorites,polarity,W1 Score,Bull_ratio,W Score With Bull Ratio,...,BinaryOBV,Binary20 period CCI,Binary14 period EMV,BinaryVIm,BinaryVIp,Binaryema50,Binaryema21,Binaryema15,Binaryema5,Actual_Label
0,4.789144,0.764092,32454.039666,2574.079332,60.407159,6.317328,0.111223,0.016004,4.386364,0.070199,...,0,0,0,0,0,0,0,0,0,1.0
1,9.122449,1.253968,18279.126984,2235.809524,69.537416,11.630385,0.101236,0.180223,5.379310,0.969473,...,0,0,0,0,0,0,0,0,0,1.0
2,1.325521,0.289062,5000.981771,2029.653646,31.230965,1.903646,0.100770,0.096907,4.323529,0.418980,...,0,0,0,0,0,0,0,0,0,0.0
3,2.045699,0.303763,9224.862903,3983.379032,113.469624,2.653226,0.113049,0.042449,4.656250,0.197655,...,0,0,0,0,0,1,1,1,1,1.0
4,0.869674,0.102757,14964.082707,2214.571429,79.086570,1.075188,0.081205,0.007195,4.379310,0.031511,...,0,1,0,0,0,0,0,0,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25531,3.011905,0.392857,14180.595238,1765.121429,31.755382,3.797619,0.095683,0.135134,4.277778,0.578073,...,1,1,0,1,0,1,1,1,1,0.0
25532,14.400000,2.407143,16161.992857,2084.061905,182.260151,19.214286,0.082271,0.107210,3.648649,0.391172,...,1,1,1,0,1,1,1,1,1,1.0
25533,21.570806,3.270153,11254.296296,1814.954248,428.484448,28.111111,0.070485,0.165014,1.887324,0.311435,...,0,1,1,0,1,0,0,0,0,0.0
25534,7.505967,0.878282,10556.818616,2143.973747,140.745063,9.262530,0.079407,0.135342,3.945946,0.534052,...,1,1,1,0,1,1,1,1,1,1.0


In [10]:
Df_hourly_merge2 =Df_hourly_merge2.dropna()

In [11]:
Df_hourly_merge2

,favorites,retweets,number_of_followers,following,followers_following_ratio,2x_retweets_+_favorites,polarity,W1 Score,Bull_ratio,W Score With Bull Ratio,...,BinaryOBV,Binary20 period CCI,Binary14 period EMV,BinaryVIm,BinaryVIp,Binaryema50,Binaryema21,Binaryema15,Binaryema5,Actual_Label
14,3.807018,0.864662,9089.250627,2538.889724,18.931744,5.536341,0.097035,0.089301,5.071429,0.452885,...,1,1,0,0,0,1,1,1,1,1.0
15,1.249249,0.453453,13636.003003,3120.468468,155.932786,2.156156,0.083099,0.101788,3.000000,0.305364,...,0,1,0,0,0,0,0,0,0,0.0
16,1.976905,0.360277,12363.824480,4591.815242,92.103378,2.697460,0.107076,0.081716,6.777778,0.553850,...,1,1,1,0,1,1,1,1,1,0.0
17,1.244306,0.254658,12788.830228,5277.569358,38.833412,1.753623,0.117566,0.077150,3.280702,0.253107,...,1,1,1,0,1,1,1,1,1,1.0
18,2.364896,0.461894,19970.196305,5026.699769,61.380070,3.288684,0.107411,0.098648,3.574468,0.352613,...,0,0,0,1,0,0,0,0,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25530,5.783333,0.659524,12129.000000,1975.276190,108.150291,7.102381,0.077630,0.078660,3.022222,0.237728,...,1,0,0,1,0,1,1,1,1,0.0
25531,3.011905,0.392857,14180.595238,1765.121429,31.755382,3.797619,0.095683,0.135134,4.277778,0.578073,...,1,1,0,1,0,1,1,1,1,0.0
25532,14.400000,2.407143,16161.992857,2084.061905,182.260151,19.214286,0.082271,0.107210,3.648649,0.391172,...,1,1,1,0,1,1,1,1,1,1.0
25533,21.570806,3.270153,11254.296296,1814.954248,428.484448,28.111111,0.070485,0.165014,1.887324,0.311435,...,0,1,1,0,1,0,0,0,0,0.0


# Trying LTSM 

In [12]:
Df_hourly_merge3= Df_hourly_merge2.copy()

In [13]:
Df_hourly_merge3

,favorites,retweets,number_of_followers,following,followers_following_ratio,2x_retweets_+_favorites,polarity,W1 Score,Bull_ratio,W Score With Bull Ratio,...,BinaryOBV,Binary20 period CCI,Binary14 period EMV,BinaryVIm,BinaryVIp,Binaryema50,Binaryema21,Binaryema15,Binaryema5,Actual_Label
14,3.807018,0.864662,9089.250627,2538.889724,18.931744,5.536341,0.097035,0.089301,5.071429,0.452885,...,1,1,0,0,0,1,1,1,1,1.0
15,1.249249,0.453453,13636.003003,3120.468468,155.932786,2.156156,0.083099,0.101788,3.000000,0.305364,...,0,1,0,0,0,0,0,0,0,0.0
16,1.976905,0.360277,12363.824480,4591.815242,92.103378,2.697460,0.107076,0.081716,6.777778,0.553850,...,1,1,1,0,1,1,1,1,1,0.0
17,1.244306,0.254658,12788.830228,5277.569358,38.833412,1.753623,0.117566,0.077150,3.280702,0.253107,...,1,1,1,0,1,1,1,1,1,1.0
18,2.364896,0.461894,19970.196305,5026.699769,61.380070,3.288684,0.107411,0.098648,3.574468,0.352613,...,0,0,0,1,0,0,0,0,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25530,5.783333,0.659524,12129.000000,1975.276190,108.150291,7.102381,0.077630,0.078660,3.022222,0.237728,...,1,0,0,1,0,1,1,1,1,0.0
25531,3.011905,0.392857,14180.595238,1765.121429,31.755382,3.797619,0.095683,0.135134,4.277778,0.578073,...,1,1,0,1,0,1,1,1,1,0.0
25532,14.400000,2.407143,16161.992857,2084.061905,182.260151,19.214286,0.082271,0.107210,3.648649,0.391172,...,1,1,1,0,1,1,1,1,1,1.0
25533,21.570806,3.270153,11254.296296,1814.954248,428.484448,28.111111,0.070485,0.165014,1.887324,0.311435,...,0,1,1,0,1,0,0,0,0,0.0


In [14]:
Df_hourly_merge3.shape

(25386, 84)

In [24]:
%%time


Df_hourly_merge3= Df_hourly_merge2.copy()
# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
Df_hourly_merge3 = scaler.fit_transform(Df_hourly_merge3)

#Saveoverall results
OVERAL_LTSM_RESULTS = []

#Number of timesteps to check
for time in [2,6,12,24,48,96,168,336]:
    # Define the number of timesteps and features
    timesteps = time
    features = 83
    print(" ")
    print(" ")
    print("TESTING TIMESTEP " + str(time))
    print(" ")
    # Create the input and output sequences
    X = []
    y = []

    for i in range(timesteps, Df_hourly_merge3.shape[0]): 
        X.append(Df_hourly_merge3[i-timesteps:i, :-1]) #All features except last column
        y.append(Df_hourly_merge3[i, -1]) #Label column
    X, y = np.array(X), np.array(y)

    # Reshape the data to match the input shape of the LSTM model
    X = np.reshape(X, (X.shape[0], X.shape[1], features))
    
    
    # Split the data using TimeSeriesSplit
    tscv = TimeSeriesSplit(n_splits=10)


    LTSM_RESULTS = []
    #Number of neurons to check
    for neurons in [2,8,32,65, 128, 265]:
        for train_index, test_index in tscv.split(X):
            print(train_index, test_index)

            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            #Dropurate to check
            for dropoutnum in [0.1,0.2,0.3,0.4]:
                # Define the model
                model = Sequential()
                model.add(LSTM(neurons, return_sequences=True, input_shape=(X_train.shape[1], features)))
                # model.add(LSTM(100, return_sequences=True))
                # model.add(LSTM(100, return_sequences=True))
                model.add(LSTM(neurons,dropout = dropoutnum))
                model.add(Dense(1, activation='sigmoid'))
                
                #Learningrate to check
                for learningrate in [0.1,0.01,0.001]:
                    # Compile the model
                    optimizer = Adam(learning_rate=learningrate)
                    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

                    # Fit the model to the training data
                    history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test), verbose=2)

                    # Make predictions
                    y_pred = model.predict(X_test)
                    testy_pred = (y_pred >= 0.5).astype(int)
                    LTSMaccuracy = accuracy_score(y_test, testy_pred)

                    LTSM_RESULTS.append(LTSMaccuracy)

                    # Print classification report
                    print(classification_report(y_test, testy_pred))
                    print(confusion_matrix(y_test, testy_pred))
                    print(" ")
                    print('LTSM Accuracy = ' + str( sum(LTSM_RESULTS) / len(LTSM_RESULTS))+ '; Timestep = ' + str(time)  + '; Neuron =' +str(neurons) + '; Dropout =' +str(dropoutnum)+'; Learning =' +str(learningrate))
                    print(" ")
                    OVERAL_LTSM_RESULTS.append([sum(LTSM_RESULTS) / len(LTSM_RESULTS), time, neurons, dropoutnum, learningrate])
   



 
 
TESTING TIMESTEP 2
 
[   0    1    2 ... 2311 2312 2313] [2314 2315 2316 ... 4618 4619 4620]
Epoch 1/10
73/73 - 4s - loss: 0.6950 - accuracy: 0.4927 - val_loss: 0.6920 - val_accuracy: 0.5314 - 4s/epoch - 52ms/step
Epoch 2/10
73/73 - 1s - loss: 0.6936 - accuracy: 0.5246 - val_loss: 0.6918 - val_accuracy: 0.5314 - 824ms/epoch - 11ms/step
Epoch 3/10
73/73 - 1s - loss: 0.6924 - accuracy: 0.5324 - val_loss: 0.6912 - val_accuracy: 0.5314 - 819ms/epoch - 11ms/step
Epoch 4/10
73/73 - 1s - loss: 0.6949 - accuracy: 0.5151 - val_loss: 0.6924 - val_accuracy: 0.5314 - 788ms/epoch - 11ms/step
Epoch 5/10
73/73 - 1s - loss: 0.6934 - accuracy: 0.5151 - val_loss: 0.6947 - val_accuracy: 0.5314 - 804ms/epoch - 11ms/step
Epoch 6/10
73/73 - 1s - loss: 0.6949 - accuracy: 0.5186 - val_loss: 0.6915 - val_accuracy: 0.5314 - 828ms/epoch - 11ms/step
Epoch 7/10
73/73 - 1s - loss: 0.6929 - accuracy: 0.5290 - val_loss: 0.6914 - val_accuracy: 0.5314 - 888ms/epoch - 12ms/step
Epoch 8/10
73/73 - 1s - loss: 0.6932 -

C:\Users\mario\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\mario\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\mario\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


73/73 - 4s - loss: 0.6917 - accuracy: 0.5324 - val_loss: 0.6912 - val_accuracy: 0.5314 - 4s/epoch - 53ms/step
Epoch 2/10
73/73 - 1s - loss: 0.6913 - accuracy: 0.5324 - val_loss: 0.6912 - val_accuracy: 0.5314 - 798ms/epoch - 11ms/step
Epoch 3/10
73/73 - 1s - loss: 0.6914 - accuracy: 0.5324 - val_loss: 0.6912 - val_accuracy: 0.5314 - 863ms/epoch - 12ms/step
Epoch 4/10
73/73 - 1s - loss: 0.6911 - accuracy: 0.5324 - val_loss: 0.6912 - val_accuracy: 0.5314 - 901ms/epoch - 12ms/step
Epoch 5/10
73/73 - 1s - loss: 0.6912 - accuracy: 0.5324 - val_loss: 0.6913 - val_accuracy: 0.5314 - 894ms/epoch - 12ms/step
Epoch 6/10
73/73 - 1s - loss: 0.6916 - accuracy: 0.5324 - val_loss: 0.6912 - val_accuracy: 0.5314 - 896ms/epoch - 12ms/step
Epoch 7/10
73/73 - 1s - loss: 0.6914 - accuracy: 0.5324 - val_loss: 0.6912 - val_accuracy: 0.5314 - 843ms/epoch - 12ms/step
Epoch 8/10
73/73 - 1s - loss: 0.6913 - accuracy: 0.5324 - val_loss: 0.6912 - val_accuracy: 0.5314 - 830ms/epoch - 11ms/step
Epoch 9/10
73/73 - 1s 

C:\Users\mario\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\mario\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\mario\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


73/73 - 4s - loss: 0.6911 - accuracy: 0.5324 - val_loss: 0.6912 - val_accuracy: 0.5314 - 4s/epoch - 57ms/step
Epoch 2/10


KeyboardInterrupt: 

In [22]:
OVERAL_LTSM_RESULTS

[[0.46857390550498484, 2, 2, 0.1, 0.1],
 [0.5, 2, 2, 0.1, 0.01],
 [0.5104753648316717, 2, 2, 0.1, 0.001],
 [0.5157130472475075, 2, 2, 0.2, 0.1],
 [0.5188556566970091, 2, 2, 0.2, 0.01],
 [0.5209507296633434, 2, 2, 0.2, 0.001],
 [0.5134683262121493, 2, 2, 0.3, 0.1],
 [0.5157130472475075, 2, 2, 0.3, 0.01],
 [0.5174589413861195, 2, 2, 0.3, 0.001],
 [0.5188556566970091, 2, 2, 0.4, 0.1],
 [0.5199984237695551, 2, 2, 0.4, 0.01],
 [0.5209507296633434, 2, 2, 0.4, 0.001],
 [0.5189223433696776, 2, 2, 0.1, 0.1],
 [0.5171837265465353, 2, 2, 0.1, 0.01],
 [0.5156769252998121, 2, 2, 0.1, 0.001],
 [0.5150357607282184, 2, 2, 0.2, 0.1],
 [0.513832581146893, 2, 2, 0.2, 0.01],
 [0.5127630881857148, 2, 2, 0.2, 0.001],
 [0.5118061734309766, 2, 2, 0.3, 0.1],
 [0.510944950151712, 2, 2, 0.3, 0.01],
 [0.5101657481371393, 2, 2, 0.3, 0.001],
 [0.509457382669346, 2, 2, 0.4, 0.1],
 [0.5088106141987521, 2, 2, 0.4, 0.01],
 [0.5082177431007077, 2, 2, 0.4, 0.001],
 [0.5079843953185953, 2, 2, 0.1, 0.1],
 [0.50776899736587

In [ ]:
ResultsLTSMDF = pd.DataFrame(OVERAL_LTSM_RESULTS, columns  = ['LSTM Acc.', 'Timestep', 'Neurons', 'Dropout', 'Learning'])

In [ ]:
ResultsLTSMDF

In [ ]:
ResultsLTSMDF.to_csv('assets/testingresults.csv')